# Question 2 - VaR Calculations (Linear and Quadratic Approximations)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd

## Initialize Variables

In [114]:
#Call option
So1= 50         #Intial Value for stock 1
K1 = 51         #Strike Price 
sig1 = 0.28     #Volatility 
T1 = 9/12       #Time to Maturity

#Put Option
So2= 20         #Intial Value for stock 1
K2 = 19         #Strike Price 
sig2 = 0.25     #Volatility 
T1 = 1          #Time to Maturity 

r = 0.06        #Risk free rate
rho = 0.4       #
N =10#100000       #Simulations 


## Main Code Block 

### Part a 